In [87]:
import torch 
import chromadb_handler as CH
import pandas as pd

In [89]:
import PyPDF2
import json
from pathlib import Path
import os

# turn true if rerun of data
cast_data = True

if cast_data:
    directory_str = "data/DUUIDataset/training"
    directory = os.fsencode(directory_str)
    counter = 1    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        print(filename)
        if filename.endswith(".pdf") : 
            PDF_PATH = directory_str + "/" + filename
            CHUNK_SIZE = 300  # Anzahl Zeichen, ≈ 500–700 Tokens

            # -----------------------------
            # 1. PDF Laden
            # -----------------------------
            pdf_reader = PyPDF2.PdfReader(PDF_PATH)

            # Metadaten aus PDF
            meta = pdf_reader.metadata

            title = meta.title if meta and meta.title else "Unknown Title"
            authors = meta.author.split(",") if meta and meta.author else []

            # -----------------------------
            # 2. Seiteninhalt extrahieren
            # -----------------------------
            pages = []
            for i, page in enumerate(pdf_reader.pages):
                text = page.extract_text()
                pages.append({"page": i+1, "text": text})

            # -----------------------------
            # 3. Chunks bauen
            # -----------------------------
            def chunk_text(text, size=CHUNK_SIZE):
                chunks = []
                for i in range(0, len(text), size):
                    chunks.append(text[i:i+size])
                return chunks

            dataset = []
            doc_id = Path(PDF_PATH).stem

            for page in pages:
                text = page["text"]
                if not text:
                    continue

                chs = chunk_text(text)

                for idx, chunk in enumerate(chs):
                    entry = {
                        "id": doc_id,
                        "chunk_id": f"{doc_id}_p{page['page']}_c{idx}",
                        "source": PDF_PATH,
                        "title": title,
                        "authors": authors,
                        "publication_year": "None",
                        "page_start": page["page"],
                        "page_end": page["page"],
                        "text": chunk,
                        "embedding": None
                    }
                    dataset.append(entry)

            # -----------------------------
            # 4. JSON exportieren
            # -----------------------------
            with open(f"rag_dataset_{counter}.jsonl", "w", encoding="utf-8") as f:
                for item in dataset:
                    f.write(json.dumps(item, ensure_ascii=False) + "\n")
            counter += 1
            print("FERTIG! Datei rag_dataset.jsonl generiert.")



            continue
        else:
            continue

rag_dataset_4.jsonl
2023.findings-emnlp.29.pdf
FERTIG! Datei rag_dataset.jsonl generiert.
rag_dataset_1.jsonl
rag_dataset_2.jsonl
2025.konvens-1.22.pdf
FERTIG! Datei rag_dataset.jsonl generiert.
rag_dataset_3.jsonl
1-s2.0-S2352711024004047-main.pdf
FERTIG! Datei rag_dataset.jsonl generiert.
BAUMARTZ_Daniel_DUUI__A_Toolbox_for_the_Construction_of_a_ne.pdf
FERTIG! Datei rag_dataset.jsonl generiert.


In [92]:
import chromadb

ch = CH.chromaDBWrapper()
client = chromadb.PersistentClient("chroma")
client.delete_collection(name="DUUI_300")
client.get_or_create_collection(name="DUUI_300")
collection = client.get_collection("DUUI_300")


In [97]:
# Insert JSONL data into the ChromaDB

import PyPDF2
import json
from pathlib import Path
import os

# turn true if rerun of data
cast_data = True

embedding_structure = []
ids_fl = []  
metadatas_fl = []
documents_fl = []
uris_fl = []
if cast_data:
    directory_str = "data/DUUIDataset/training"
    directory = os.fsencode(directory_str)
    counter = 1  
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        str_filename = directory_str + "/" + filename
        print(str_filename)
        if filename.endswith(".jsonl") : 
            with open(str_filename) as f:
                data = [json.loads(line) for line in f]
                for line in data:
                    #ids
                    ids_fl.append(line["chunk_id"])
                    # metadeta
                    if isinstance(authors, list):
                        authors = ", ".join(authors)
                    metadeta_dict = {"title": line["title"], 
                                     "authors": authors, 
                                     "publication_year": line["publication_year"], 
                                     "page_start": line["page_start"], 
                                     "page_end": line["page_end"]}
                    metadatas_fl.append(metadeta_dict)
                    # Docuemnts / text
                    documents_fl.append(line["text"])
                    # Uris
                    uris_fl.append(line["source"])
            #cast JSON arugment into proper Form


            continue
        else:
            continue
#add metadatas
collection.add(ids=ids_fl, metadatas=metadatas_fl, documents=documents_fl)

data/DUUIDataset/training/rag_dataset_4.jsonl
data/DUUIDataset/training/2023.findings-emnlp.29.pdf
data/DUUIDataset/training/rag_dataset_1.jsonl
data/DUUIDataset/training/rag_dataset_2.jsonl
data/DUUIDataset/training/2025.konvens-1.22.pdf
data/DUUIDataset/training/rag_dataset_3.jsonl
data/DUUIDataset/training/1-s2.0-S2352711024004047-main.pdf
data/DUUIDataset/training/BAUMARTZ_Daniel_DUUI__A_Toolbox_for_the_Construction_of_a_ne.pdf


In [104]:
df = pd.read_csv("data/BBCNews/bbc_news.csv")[:10000]
df.head()

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...


In [ ]:
import pandas as pd
from urllib.parse import urlparse
import hashlib
from datetime import datetime

df = pd.read_csv("data/BBCNews/bbc_news.csv")[:5000]
df = df.sample(n=5000, random_state=42) 
# set properties

ids_b = []
metadatas_b = []
docuemnts_b = []
unique_set = set()
for row in df.itertuples(index=False):
    # Create a short hash from full URL
    # form year_shortURl_Hash of whole URL
    # create unique ID
    domain = urlparse(row.link).netloc.replace(".", "_")
    hash_part = hashlib.md5((row.link+row.title).encode("utf-8")).hexdigest()[:16]
    date = row.pubDate
    dt = datetime.strptime(date, "%a, %d %b %Y %H:%M:%S %Z")
    id = hash_part+"_"+str(dt.year)
    # text/documents
    document = row.description
    #metadata
    metadeta_dict = {"title": row.title, 
                     "authors": authors, 
                                     "publication_date": row.pubDate, 
                                     "link" : row.link,
                                     "guide": row.guid}

    # check if id (hash of the )
    if id not in unique_set:
        unique_set.add(id)
        # if not in unique set
        metadatas_b.append(metadeta_dict)
        docuemnts_b.append(row.description)
        ids_b.append(id)
    
collection_b = client.get_or_create_collection(name="bbc_news")
collection_b.add(ids=ids_b, metadatas=metadatas_b, documents=docuemnts_b)

InternalError: ValueError: Batch size of 9546 is greater than max batch size of 5461

In [ ]:
df = pd.read_csv("data/BBCNews/bbc_news.csv")[:10000]
output_file = open("output_LLM.txt", "w", encoding="utf-8")
for row in df.itertuples(index=False):
    output_file.write(f"{row.description} \n")
output_file.close()

In [ ]:
date_str = "Mon, 07 Mar 2022 08:01:56 GMT"
dt = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %Z")